In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import json

def login(driver, site_info):
    """
    주어진 사이트 정보에 따라 로그인을 수행합니다.

    Args:
        driver: Selenium WebDriver 인스턴스.
        site_info: JSON 파일에서 로드된 사이트 정보 (딕셔너리).

    Returns:
        로그인 성공 여부 (Boolean).  True: 성공, False: 실패
    """
    try:
        driver.get(site_info["login_url"])

        # ID 입력
        id_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, site_info["id_selector"]))
        )
        id_input.send_keys(site_info["id"])

        # PW 입력
        pw_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, site_info["pw_selector"]))
        )
        pw_input.send_keys(site_info["pw"])

        # 로그인 버튼 클릭
        login_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, site_info["login_button_selector"]))
        )
        login_button.click()

        # 로그인 성공 여부 확인 (예:  특정 요소가 로딩될 때까지 기다림)
        #   예:  마이페이지 링크,  주문 내역 테이블 등.  사이트에 맞게 수정 필요
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#dometopia_header > div.header-top-wrap > div.topmenu > ul > li:nth-child(3) > a"))  # 예시: 마이페이지 링크
        )
        return True

    except Exception as e:
        print(f"로그인 중 오류 발생: {e}")
        return False

def get_order_info(driver, site_info):
    """
    주문 정보를 가져옵니다. (로그인 후 수행되는 함수)
    """

    order_info_list = []

    for order_list in site_info["order_lists"]:
        # 네비게이션 단계 수행
        for step in order_list["navigation_steps"]:
            if step["action"] == "click":
                element = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, step["selector"]))
                )
                element.click()

            # 다른 액션 (예: "wait") 추가 가능
            elif step["action"] == "wait":
                time.sleep(step["duration"])

        # 주문 번호 가져오기 (여러 개일 수 있으므로 리스트 처리)
        order_numbers = []
        try:
            order_number_elements = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, order_list["order_number_selector"]))
            )
            order_numbers = [elem.text.strip() for elem in order_number_elements]
        except Exception as e:
            print(f"주문 번호 가져오기 오류: {e}")
            continue # 다음 order_list로 넘어감


        # 각 주문 번호에 대해 상세 정보 가져오기
        for order_number in order_numbers:

            # 주문 번호 클릭해서 상세 페이지로 이동
            try:
                order_number_link = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.LINK_TEXT, order_number))
                )
                order_number_link.click()
            except Exception as e:
                print(f"주문번호 클릭 오류: {e}")
                continue

            # 상세 정보 가져오기
            try:
                 # 배송 업체
                delivery_company_element = WebDriverWait(driver, 20).until(  # 더 긴 시간 대기
                    EC.presence_of_element_located((By.CSS_SELECTOR, order_list["delivery_company_selector"]))
                )
                delivery_company = delivery_company_element.text.strip()

                # 이름 가져오기
                name_element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, order_list["name_selector"]))
                )
                name = name_element.text.strip()

                order_info = {
                    "order_number": order_number,
                    "delivery_company": delivery_company,
                    "name" : name
                }
                order_info_list.append(order_info)

            except Exception as e:
                 print(f"{order_number} 상세 정보 가져오기 오류: {e}")
                 continue # 다음 order_number로

            # 다시 목록 페이지로 돌아가기 (back)
            driver.back()
            # *중요*:  back() 후에는  DOM이 변경되므로,  order_number_elements를 다시 찾아야 함 (StaleElementReferenceException 방지)
            #         위의  order_number_elements = ...  라인을  for loop 안,  driver.back()  바로 다음에 넣어주면 됨.
            order_number_elements = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, order_list["order_number_selector"]))
            )


    return order_info_list


if __name__ == "__main__":
    # JSON 파일 로드
    json_file_path = r"C:\Users\jh\택배송장\site_configs\dmtopia_config.json"  # 절대 경로 사용
    with open(json_file_path, "r", encoding="utf-8") as f:
        site_info = json.load(f)

    # WebDriver 설정 (Chrome 사용 예시)
    driver = webdriver.Chrome()

    # 로그인 시도
    if login(driver, site_info):
        print("로그인 성공!")

        # 주문 정보 가져오기
        order_info = get_order_info(driver, site_info)

        # 결과 출력
        if order_info:
             print("주문 정보:")
             for info in order_info:
                print(info)
        else:
            print("주문 정보를 찾을 수 없습니다.")

    else:
        print("로그인 실패.")


    # WebDriver 종료 (마지막에)
    driver.quit()

로그인 성공!
주문 번호 가져오기 오류: Message: 

주문 정보를 찾을 수 없습니다.
